### Criar os diretórios

In [0]:
# Configurações iniciais
catalog = "workspace"
schemas = ["processar", "processado", "bronze", "silver", "gold"]
volume = "vendas"
# Criar catálogo (se não existir)
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
# Criar schemas e volumes dentro do catálogo
for schema in schemas:
    # Criar schema
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}")
 
    # Criar volume
    spark.sql(f"CREATE VOLUME IF NOT EXISTS {catalog}.{schema}.{volume}")
 
    print(f"Volume criado: {catalog}.{schema}.{volume}")

In [0]:
# Imports necessários
import requests
from pyspark.dbutils import DBUtils
# Configurações
url = "https://github.com/andrerosa77/trn-pyspark/raw/main/dados_2012.csv"
final_path = "/Volumes/workspace/processar/vendas/dados_2012.csv"
temp_path = "/Volumes/workspace/temp/vendas/dados_2012.csv"
# Inicializar DBUtils
dbutils = DBUtils(spark)
# 1. Criar volume temporário (se não existir)
spark.sql("CREATE CATALOG IF NOT EXISTS workspace")
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.temp")
spark.sql("CREATE VOLUME IF NOT EXISTS workspace.temp.vendas")
 
print("Volume temporário criado: workspace.temp.vendas")
# 2. Fazer download do arquivo
print("📥 Fazendo download do arquivo...")
response = requests.get(url)
response.raise_for_status()
# 3. Salvar conteúdo diretamente no volume temporário
content_str = response.content.decode('utf-8')
dbutils.fs.put(temp_path, content_str, overwrite=True)
 
print(f"Arquivo temporário salvo em: {temp_path}")
# 4. Mover para o destino final
dbutils.fs.mv(temp_path, final_path)
print(f"Arquivo movido para destino final: {final_path}")
# 5. Limpar arquivo temporário (opcional)
dbutils.fs.rm(temp_path)
print("🧹 Arquivo temporário removido")
# 6. Verificar conteúdo do volume de destino
print("\n📂 Conteúdo do volume de destino:")
display(dbutils.fs.ls("/Volumes/workspace/processar/vendas"))